In [2]:
import pandas as pd
import glob
import numpy as np
import math
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import statsmodels
from skbio.stats.ordination import pcoa 
from scipy.spatial import distance
from matplotlib.lines import Line2D
from sklearn.metrics import confusion_matrix
import pyreadr
import warnings
from scipy.spatial.distance import braycurtis
from scipy.spatial.distance import pdist, squareform
from skbio.stats.ordination import pcoa 
from sklearn.metrics import precision_score, recall_score
warnings.filterwarnings("ignore")
from skbio.stats.composition import clr
from scipy.stats import wilcoxon
from statsmodels.stats import multitest

### WILCOXON PAIRED TEST

In [3]:
def to_clr(df, pseudocount_vector):
    ''' data - features as index and samples as columns '''
    
    df = df.div(df.sum(axis=1), axis=0)
    data = df + pseudocount_vector.reshape(len(pseudocount_vector), 1)
    data = data.T.copy()
    #data += 1e-2                                  # add pseudocount
    return pd.DataFrame(clr(data.T), columns=data.index, index=data.columns)

In [8]:
def run_wilcoxon_test(i, N, sampling, transformation='clr'):
    
    sim_dir = "/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/simulation/efs3/"

    counts = pyreadr.read_r(sim_dir + f"rep{i}/counts_n{N}.rds")[None]
    metadata = pyreadr.read_r(sim_dir + f"rep{i}/metadata_n{N}.rds")[None]
    gt_file = pd.read_csv(sim_dir + f'rep{i}/gt_features.tsv', sep='\t')

    
    filtered_metadata = metadata[metadata['delta.t'].isin(sampling)].copy()
    filtered_metadata['intervention'] = np.where((filtered_metadata['delta.t'] > 9) & (filtered_metadata['delta.t'] < 20), 0, 1)
    filtered_counts = counts[filtered_metadata['sample_name']].T
    pseudocount_vector = (filtered_counts[filtered_counts>0].min(axis=1) * 1e-1).values

    if transformation == 'log_relab':
        input_counts = np.log(filtered_counts + 1e-2)
    elif transformation == 'clr':
        input_counts = to_clr(filtered_counts, pseudocount_vector)

    merged_df = pd.merge(input_counts, filtered_metadata.set_index('sample_name'), left_index=True, right_index=True)
    features = filtered_counts.columns

    pvalues_list = []
    for feature in features:
        df = merged_df[[feature, 'host_subject_id', 'delta.t', 'Group', 'intervention']]
        df['host_subject_id_new'] = df['host_subject_id'] + ['_A', '_B', '_C', '_D'] * int(len(df)/4)
        pivot_df = df.pivot(index='host_subject_id_new', columns='intervention', values=feature)

        if pivot_df.columns.isin([1, 0]).all() and pivot_df[[1, 0]].dropna().shape[0] > 1:
            stat, p_value = wilcoxon(pivot_df[1].dropna(), pivot_df[0].dropna(), zero_method='zsplit', alternative='two-sided')
            pvalues_list.append({'feature': feature, 'pvalue': p_value})
        else:
            pvalues_list.append({'feature': feature, 'pvalue': None})

    results_df = pd.DataFrame(pvalues_list)
    results_df = pd.merge(results_df.set_index('feature'), gt_file.set_index('feature'), left_index=True, right_index=True)
    results_df['qval'] = multitest.multipletests(results_df['pvalue'].dropna(), method='fdr_bh')[1]
    results_df['ypred'] = np.where(results_df['qval'] < 0.05, 1, 0)
    results_df['ytrue'] = np.abs(results_df['upregulated'])
    tn, fp, fn, tp = confusion_matrix(results_df['ytrue'], results_df['ypred']).ravel()
    precision = precision_score(results_df['ytrue'], results_df['ypred'], zero_division=0)
    recall = recall_score(results_df['ytrue'], results_df['ypred'])

    res = [{
        'sample_size': N,
        'rep': i,
        'sampling': len(sampling),
        'transformation': transformation,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
        'ypred':results_df['ypred'].sum(),
        'recall': recall,
        'precision': precision
    }]
    
    results_df['FP'] = np.where((results_df.ytrue == 0) & (results_df.ypred == 1), 1, 0)
    results_df.to_csv(f'/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/wilcoxon_all/{transformation}/wilcoxon_sampling_{N}_efs_3_rep_{i}_d_{len(sampling)}.csv')
    
    return pd.DataFrame(res)

In [3]:
def run_wilcoxon_test(i, N, sampling, transformation='clr'):
    
    sim_dir = "/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/simulation/efs3/"

    counts = pyreadr.read_r(sim_dir + f"rep{i}/counts_n{N}.rds")[None]
    metadata = pyreadr.read_r(sim_dir + f"rep{i}/metadata_n{N}.rds")[None]
    gt_file = pd.read_csv(sim_dir + f'rep{i}/gt_features.tsv', sep='\t')

    
    filtered_metadata = metadata[metadata['delta.t'].isin(sampling)].copy()
    filtered_metadata['intervention'] = np.where((filtered_metadata['delta.t'] > 9) & (filtered_metadata['delta.t'] < 20), 0, 1)
    filtered_counts = counts[filtered_metadata['sample_name']].T
    pseudocount_vector = (filtered_counts[filtered_counts>0].min(axis=1) * 1e-1).values

    if transformation == 'log_relab':
        input_counts = np.log(filtered_counts + 1e-2)
    elif transformation == 'clr':
        input_counts = to_clr(filtered_counts, pseudocount_vector)

    merged_df = pd.merge(input_counts, filtered_metadata.set_index('sample_name'), left_index=True, right_index=True)
    features = filtered_counts.columns

    pvalues_list = []
    for feature in features:
        df = merged_df[[feature, 'host_subject_id', 'delta.t', 'Group', 'intervention']]
        df['host_subject_id_new'] = df['host_subject_id'] + ['_A', '_B', '_C', '_D'] * int(len(df)/4)
        pivot_df = df.pivot(index='host_subject_id_new', columns='intervention', values=feature)

        if pivot_df.columns.isin([1, 0]).all() and pivot_df[[1, 0]].dropna().shape[0] > 1:
            stat, p_value = wilcoxon(pivot_df[1].dropna(), pivot_df[0].dropna(), zero_method='zsplit', alternative='two-sided')
            pvalues_list.append({'feature': feature, 'pvalue': p_value})
        else:
            pvalues_list.append({'feature': feature, 'pvalue': None})

    results_df = pd.DataFrame(pvalues_list)
    #results_df = pd.merge(results_df.set_index('feature'), gt_file.set_index('feature'), left_index=True, right_index=True)
    #results_df.to_csv(f'/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/wilcoxon_all/{transformation}/wilcoxon_sampling_{N}_efs_3_rep_{i}_d_{len(sampling)}.csv')
    
    return merged_df#results_df#pd.DataFrame(res)

In [4]:
import numpy as np
import pandas as pd
import pyreadr
from scipy.stats import wilcoxon
from pathlib import Path

def process_data(i, N, sampling, transformation, sim_dir, efs):
    # Use pathlib for path handling
    base_path = Path(sim_dir) / f"rep{i}"
    
    # Load data more efficiently
    counts = pyreadr.read_r(base_path / f"counts_n{N}.rds")[None]
    metadata = pyreadr.read_r(base_path / f"metadata_n{N}.rds")[None]
    gt_file = pd.read_csv(base_path / 'gt_features.tsv', sep='\t')
    
    # Filter metadata once
    mask = metadata['delta.t'].isin(sampling)
    filtered_metadata = metadata[mask].copy()
    filtered_metadata['intervention'] = ((filtered_metadata['delta.t'] > 9) & 
                                      (filtered_metadata['delta.t'] < 20)).astype(int)
    
    # Filter counts using boolean indexing
    filtered_counts = counts[filtered_metadata['sample_name']].T
    
    # Calculate pseudocounts more efficiently
    pseudocount_vector = filtered_counts[filtered_counts > 0].min(axis=1).values * 0.1
    
    # Transform data
    if transformation == 'log_relab':
        input_counts = np.log(filtered_counts + 0.01)
    elif transformation == 'clr':
        input_counts = to_clr(filtered_counts, pseudocount_vector)
    
    # Merge data more efficiently
    merged_df = pd.concat([
        input_counts,
        filtered_metadata.set_index('sample_name')
    ], axis=1)
    
    # Calculate p-values more efficiently
    pvalues_list = []
    features = filtered_counts.columns
    
    for feature in features:
        df_subset = merged_df[[feature, 'host_subject_id']]
        
        # Vectorized operations instead of loops
        result_data = []
        for host in merged_df['host_subject_id'].unique():
            host_data = df_subset[df_subset['host_subject_id'] == host]
            
            if len(sampling) == 4:  # Ensure we have enough data points
                x1 = host_data[feature].iloc[:2].values
                x2 = host_data[feature].iloc[2:4].values
                result_data.append((x1, x2))
                
            elif len(sampling) == 2:  # Ensure we have enough data points
                x1 = host_data[feature].iloc[:1].values
                x2 = host_data[feature].iloc[1:2].values
                result_data.append((x1, x2))
            
            elif len(sampling) == 6:  # Ensure we have enough data points
                x1 = host_data[feature].iloc[:3].values
                x2 = host_data[feature].iloc[3:6].values
                result_data.append((x1, x2))
        
        if result_data:
            x1_combined = np.concatenate([x[0] for x in result_data])
            x2_combined = np.concatenate([x[1] for x in result_data])
            
            stat, p_value = wilcoxon(x1_combined, x2_combined, 
                                   zero_method='zsplit', 
                                   alternative='two-sided')
            
            pvalues_list.append({
                'feature': feature,
                'pvalue': p_value
            })
    
    results_df = pd.DataFrame(pvalues_list)
    results_df = pd.merge(results_df.set_index('feature'), gt_file.set_index('feature'), left_index=True, right_index=True)
    results_df['effect_size'] = efs
    results_df['iteration'] = i
    results_df.to_csv(f'/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/results/wilcoxon/one_arm_all/{transformation}/wilcoxon_sampling_{N}_efs_{efs}_rep_{i}_d_{len(sampling)}.csv')

    return results_df

In [14]:
#sim_dir = "/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/simulation/efs2/"
wd = '/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/results/wilcoxon/one_arm_all/'

Sampling = [[2, 15], [2, 6, 15, 19], [2, 5, 8, 12, 15, 19]]

Wilcoxon_results = pd.DataFrame()
for efs in [125, 15, 2, 3, 5]:
    sim_dir = f"/Users/zkarwowska/new_embl_folder/zeevi_dataset_v5/simulation/efs{efs}/"
    for transformation in ['clr', 'log_relab']:
            for N in [10, 20, 30, 40, 50, 60, 70, 80]:
                for sampling in Sampling:
                        for i in range(1, 11):

                            file = f'{wd}/{transformation}/wilcoxon_sampling_{N}_efs_{efs}_rep_{i}_d_{len(sampling)}.csv'
                            my_file = Path(file)
                            if my_file.is_file():
                                pass
                            else:
                                process_data(i, N, sampling, transformation, sim_dir, efs)

In [13]:
Sampling = [[2, 15], [2, 6, 15, 19]]#2, 5, 8, 12, 15, 19], 
Wilcoxon_results = pd.DataFrame()
for transformation in ['log_relab', 'clr']:
        for N in [10, 20, 30, 40, 50, 60]:
            for sampling in Sampling:
                for i in range(1, 10):
                    #try:
                    df = run_wilcoxon_test(i, N, sampling, transformation)
                    Wilcoxon_results = pd.concat([Wilcoxon_results, df])
                    #except: pass

In [ ]:
DF_all_samples = Wilcoxon_results.groupby(by = ['sample_size', 'transformation']).median().reset_index()
sns.lineplot(x = DF_all_samples.sample_size, y = DF_all_samples.precision, hue = DF_all_samples.transformation)

In [ ]:
df = Wilcoxon_results[Wilcoxon_results['transformation'] == 'clr']

for i in range(5):
    df1=df[df['rep']==i]
    sns.lineplot(data=df1, x='sample_size', y = 'precision', color='k')
    sns.scatterplot(data=df1, x='sample_size', y = 'precision', color='k')

In [ ]:
#Wilcoxon_results.to_csv('/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v4/wilcoxon_one_arm.csv')

In [ ]:
def run_wilcoxon_test(i, N, sampling, transformation='clr'):
    
    sim_dir = "/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/simulation/efs3/"

    counts = pyreadr.read_r(sim_dir + f"rep{i}/counts_n{N}.rds")[None]
    metadata = pyreadr.read_r(sim_dir + f"rep{i}/metadata_n{N}.rds")[None]
    gt_file = pd.read_csv(sim_dir + f'rep{i}/gt_features.tsv', sep='\t')

    
    filtered_metadata = metadata[metadata['delta.t'].isin(sampling)].copy()
    filtered_metadata['intervention'] = np.where((filtered_metadata['delta.t'] > 9) & (filtered_metadata['delta.t'] < 20), 0, 1)
    filtered_counts = counts[filtered_metadata['sample_name']].T
    pseudocount_vector = (filtered_counts[filtered_counts>0].min(axis=1) * 1e-1).values

    if transformation == 'log_relab':
        input_counts = np.log(filtered_counts + 1e-2)
    elif transformation == 'clr':
        input_counts = to_clr(filtered_counts, pseudocount_vector)

    merged_df = pd.merge(input_counts, filtered_metadata.set_index('sample_name'), left_index=True, right_index=True)
    features = filtered_counts.columns

    pvalues_list = []
    for feature in features:
        df = merged_df[[feature, 'host_subject_id', 'delta.t', 'Group', 'intervention']]
        grouped_df = df.groupby(['host_subject_id', 'intervention']).agg(mean=(feature, 'mean')).reset_index()
        pivot_df = grouped_df.pivot(index='host_subject_id', columns='intervention', values='mean')

        if pivot_df.columns.isin([1, 0]).all() and pivot_df[[1, 0]].dropna().shape[0] > 1:
            stat, p_value = wilcoxon(pivot_df[1].fillna(1), pivot_df[0].fillna(1), zero_method='zsplit', alternative='two-sided')
            pvalues_list.append({'feature': feature, 'pvalue': p_value})
        else:
            pvalues_list.append({'feature': feature, 'pvalue': None})

    results_df = pd.DataFrame(pvalues_list)
    results_df = pd.merge(results_df.set_index('feature'), gt_file.set_index('feature'), left_index=True, right_index=True)
    results_df['qval'] = multitest.multipletests(results_df['pvalue'].dropna(), method='fdr_bh')[1]
    results_df['ypred'] = np.where(results_df['qval'] < 0.05, 1, 0)
    results_df['ytrue'] = np.abs(results_df['upregulated'])
    tn, fp, fn, tp = confusion_matrix(results_df['ytrue'], results_df['ypred']).ravel()
    precision = precision_score(results_df['ytrue'], results_df['ypred'], zero_division=0)
    recall = recall_score(results_df['ytrue'], results_df['ypred'])

    res = [{
        'sample_size': N,
        'rep': i,
        'sampling': len(sampling),
        'transformation': transformation,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
        'ypred':results_df['ypred'].sum(),
        'recall': recall,
        'precision': precision
    }]
    
    results_df['FP'] = np.where((results_df.ytrue == 0) & (results_df.ypred == 1), 1, 0)
    results_df.to_csv(f'/Users/zkarwowska/Desktop/EMBL_project/zeevi_dataset_v5/wilcoxon/{transformation}/wilcoxon_sampling_{N}_efs_3_rep_{i}_d_{len(sampling)}.csv')

    return pd.DataFrame(res)

In [ ]:
Sampling = [[2, 5, 8, 12, 15, 19], [2, 10], [2, 6, 15, 19]]
SampleSize = [10, 20, 30, 40, 50, 60]

Wilcoxon_results_mean = pd.DataFrame()
for transformation in ['clr', 'log_relab']:
    for sampling in Sampling:
        for N in SampleSize:
            for i in range(1, 10):
                df = run_wilcoxon_test(i, N, sampling, transformation)
                Wilcoxon_results_mean = pd.concat([Wilcoxon_results_mean, df])

In [ ]:
DF_G = Wilcoxon_results_mean.groupby(by = ['sample_size', 'transformation', 'sampling']).mean().reset_index()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

for i, sampling in zip(range(3), (2, 4, 6)):
    
    df = Wilcoxon_results_mean[Wilcoxon_results_mean['sampling'] == sampling]
    sns.lineplot(x = df.sample_size, y = df.precision, hue = df.transformation, ax=axes[i], lw=2)
    axes[i].title.set_text(f'F={sampling}')
    axes[i].set_ylim([-0.1, 1.1])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

for i, sampling in zip(range(3), (2, 4, 6)):
    
    df = DF_G[DF_G['sampling'] == sampling]
    sns.lineplot(x = df.sample_size, y = df.precision, hue = df.transformation, ax=axes[i], lw=2)
    sns.scatterplot(x = df.sample_size, y = df.precision, hue = df.transformation, legend=None, ax=axes[i])
    axes[i].title.set_text(f'F={sampling}')
    axes[i].set_ylim([-0.1, 1.1])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

for i, sampling in zip(range(3), (2, 4, 6)):
    
    df = DF_G[DF_G['sampling'] == sampling]
    sns.lineplot(x = df.sample_size, y = df.recall, hue = df.transformation, ax=axes[i], lw=2)
    sns.scatterplot(x = df.sample_size, y = df.recall, hue = df.transformation, legend=None, ax=axes[i])
    axes[i].title.set_text(f'F={sampling}')
    axes[i].set_ylim([-0.1, 1.1])

In [ ]:
DF_G = Wilcoxon_results_mean.groupby(by = ['sample_size', 'transformation']).median().reset_index()
sns.lineplot(x = DF_G.sample_size, y = DF_G.recall, hue = DF_G.transformation)
sns.scatterplot(x = DF_G.sample_size, y = DF_G.recall, hue = DF_G.transformation, legend=None)

In [ ]:
Wilcoxon_results_mean['setup'] = 'averaged_samples'
Wilcoxon_results['setup'] = 'all_samples'

DF = pd.concat([Wilcoxon_results_mean, Wilcoxon_results])
DF_G = DF.groupby(by = ['sample_size', 'transformation', 'setup']).mean().reset_index()

In [ ]:
plt.figure(figsize = (5, 5))
sns.lineplot(data=DF_G, x = 'sample_size', y = 'precision', hue = 'transformation', style = 'setup', lw=2)
sns.scatterplot(data=DF_G, x = 'sample_size', y = 'precision', hue = 'transformation', style = 'setup', s=100, marker='s', legend=False)
plt.title('COMPARISON BETWEEN AVERAGED AND ALL SAMPLES \n F=3, EFS=3')
plt.legend(edgecolor='w')
plt.tight_layout()
plt.savefig('setup_comparison.png', dpi=300)